이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ab316fa9de4d46b25066dfe942c5c8dce71a511b10beeec1507edf12ac6dab90
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
        .getOrCreate()

In [ ]:
# 애플 주식 데이터 다운로드
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csdf

: 

In [ ]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

: 

In [ ]:
# 파티션 수 확인
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

: 

In [ ]:
# 파티션 수 4개로 변경
df2 = df.repartition(4)
# 파티션 수 확인
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

: 

In [ ]:
# 파티션 수 줄이기. coalesce는 shuffle을 최소화하기 때문에 repartition보다 빠르다.
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

: 

In [ ]:
# csv로 저장
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

: 

In [ ]:
# parquet로 저장
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

: 

In [ ]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

: 

In [ ]:
!ls -tl dataOutput/

: 

In [ ]:
!ls -tl dataOutput/parquet/

: 

In [ ]:
!ls -tl dataOutput/avro/

: 

In [ ]:
!ls -tl dataOutput/json/

: 

: 